In [1]:
#r "./bin/Debug/net7.0/studies.dll"
#r "./bin/Debug/net7.0/core.fs.dll"
#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages FSharp.Data, 6.3.0 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\laimi\.nuget\packages\plotly.net.interactive\4.2.1\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [2]:
open FSharp.Data
open Plotly.NET

let trade_csv = "d:\\studies\\03_export_date_ticker_screenerid_gap_outcomes.csv"

let csv = studies.GapStudy.parseTradeOutcomes trade_csv

In [3]:
let outputSummary includeChart (summary:studies.Types.TradeSummary) = 

    printfn "%s:\n %i trades, %.2f%% win pct, %.2f%% avg gain, %.2f%% avg loss, %.2f avg gain/loss, %.2f EV" 
        summary.StrategyName summary.Total (summary.WinPct * 100m) (summary.AvgWin) (summary.AvgLoss) summary.AvgGainLoss summary.EV

    match includeChart with
    | true ->
        let chart = 
            Chart.Histogram(
                X = summary.Gains,
                NBinsX = 40,
                Name = "Gain Distribution"
            )

        chart.Display() |> ignore
    | false -> ()

In [4]:
let filterNamePairs = [
        ("All", fun (o:studies.Types.TradeOutcomeOutput.Row) -> true)
        ("New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28)
        ("Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 29)
        ("Gap ups", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp)
        ("Gap ups - new highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 28)
        ("Gap ups - top gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 29)
    ]

let createTradeSummaries outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Types.TradeOutcomeOutput.Row) -> t.Strategy) // all strategies, no filters

    filterNamePairs
    |> Seq.collect (
        fun (filterName,filter) ->

            tradesGroupedByStrategy
            |> Seq.map (
                fun (strategy, trades) ->
                    let filteredTrades = trades |> Seq.filter filter
                    let strategy = filteredTrades |> Seq.head |> fun x -> x.Strategy
                    let name = $"{filterName}, {strategy}"
                    studies.Types.TradeSummary.create name filteredTrades
            )
    )
    

let describeOutcomes includeChart tradeSummaries =
    
    tradeSummaries
    |> Seq.iter (
        fun summary -> outputSummary includeChart summary
    )
    
let highlightStrategies numberOfTopRecords tradeSummaries =
    
    let sortFunctions = [
        ("EV", fun (t:studies.Types.TradeSummary) -> t.EV)
        ("Win %", fun (t:studies.Types.TradeSummary) -> t.WinPct)
        ("Avg Gain", fun (t:studies.Types.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Top {numberOfTopRecords} by: {sortLabel}"
        tradeSummaries
        |> Seq.sortByDescending sortFunction
        |> Seq.take numberOfTopRecords
        |> Seq.iter (fun summary ->
            printfn $"{summary.StrategyName}: {sortFunction summary}"
        )
        printfn ""
        printfn ""
    )

In [5]:
let studiesDirectory = "d:\\studies"
let signalPath = "d:\\studies\\02_export_date_ticker_screenerid_gap.csv"
let priceFunc = studies.DataHelpers.getPricesFromCsv studiesDirectory

let runStrategies strategies = async {
    let! signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc
    
    return! studies.Trading.runTrades signalsWithPrices strategies
}

In [6]:
let strategies = 
        [
            studies.TradingStrategies.buyAndHoldStrategy (Some 5)
            studies.TradingStrategies.buyAndHoldStrategy (Some 15)
            studies.TradingStrategies.buyAndHoldStrategy (Some 30)
            studies.TradingStrategies.buyAndHoldStrategy (Some 60)
            studies.TradingStrategies.buyAndHoldStrategy None
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 5) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 15) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 30) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 60) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false None 5m
        ]

let outcomes = 
    strategies
    |> runStrategies
    |> Async.RunSynchronously

printfn "Finished running trades, generated:"
printfn $"{outcomes |> Seq.length} trade outcomes"

Records: 23520, dates: 423, tickers: 3164, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Ensured that data has prices
Records: 22848, dates: 412, tickers: 3153, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Executing trades...
Finished running trades, generated:
228480 trade outcomes


In [7]:
let outcomeDescription (x:studies.Types.TradeOutcomeOutput.Row) =
    printfn $"{x.Ticker} {x.Strategy}: {x.Opened.ToShortDateString()} @ {x.OpenPrice}, {x.Closed.ToShortDateString()} @ {x.ClosePrice}: profit of {x.PercentGain}"
    
let random = System.Random()

outcomes
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription


NX Buy and Hold with Stop Loss of 5.00%: 9/1/2023 @ 27.18, 11/14/2023 @ 30.25: profit of 11.295069904341427520235467260
FULC Buy and Hold 5 bars: 3/25/2022 @ 19.78, 4/1/2022 @ 23.74: profit of 20.020222446916076845298281090
UBCP Buy and Hold 30 bars with Stop Loss of 5.00%: 12/7/2022 @ 15.57, 12/12/2022 @ 14.608: profit of -6.1785484906872190109184328800
TCMD Buy and Hold 30 bars with Stop Loss of 5.00%: 2/24/2023 @ 13.78, 4/10/2023 @ 18.01: profit of 30.696661828737300435413642960
NIO Buy and Hold with Stop Loss of 5.00%: 9/9/2022 @ 18.4, 9/27/2022 @ 17.19: profit of -6.5760869565217391304347826100
SGU Buy and Hold 60 bars: 1/11/2023 @ 12.3, 4/10/2023 @ 12.68: profit of 3.0894308943089430894308943100
WNC Buy and Hold 60 bars with Stop Loss of 5.00%: 9/20/2022 @ 15.89, 12/14/2022 @ 23.91: profit of 50.471994965387035871617369410
SPB Buy and Hold 60 bars with Stop Loss of 5.00%: 12/2/2022 @ 59.95, 3/2/2023 @ 64.08: profit of 6.889074228523769808173477900
DUOL Buy and Hold 15 bars: 3/7/2

In [8]:
// inspect what's max gain 
outcomes
|> Seq.filter (fun x -> x.Strategy.Contains("with Stop Loss"))
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription


SMCI Buy and Hold with Stop Loss of 5.00%: 7/22/2022 @ 50.92, 11/14/2023 @ 282.25: profit of 454.30086410054988216810683425


In [9]:
// investigating what happened with the trade and why it went as it went
let signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc |> Async.RunSynchronously

let tickerOfInterest = "MLTX"

let signal, prices = 
    signalsWithPrices
    |> Seq.filter (fun (signal, prices) -> signal.Ticker = tickerOfInterest)
    |> Seq.head
    
let strategy = studies.TradingStrategies.buyAndHoldStrategyWithStopLoss true (Some 5) 5m

(signal, prices) |> strategy |> outcomeDescription

Records: 23520, dates: 423, tickers: 3164, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Ensured that data has prices
Records: 22848, dates: 412, tickers: 3153, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Open day for MLTX on 12/28/2022 12:00:00 AM is 12/29/2022 12:00:00 AM +00:00 with open day index of 183
Close day for MLTX on 12/28/2022 12:00:00 AM is based on Some 5 number of bars to hold and is 1/6/2023 12:00:00 AM +00:00 with close day index of 188
Close bar for MLTX on 12/28/2022 12:00:00 AM is 1/6/2023 12:00:00 AM +00:00 because close day reached
MLTX Buy and Hold 5 bars with Stop Loss of 5.00%: 12/29/2022 @ 10.22, 1/6/2023 @ 12.52: profit of 22.504892367906066536203522500


In [10]:
let tradeSummaries = outcomes |> createTradeSummaries

tradeSummaries |> describeOutcomes false

All, Buy and Hold 5 bars:
 22848 trades, 50.28% win pct, 7.40% avg gain, -6.87% avg loss, 1.08 avg gain/loss, 0.30 EV
All, Buy and Hold 15 bars:
 22848 trades, 48.04% win pct, 10.73% avg gain, -10.93% avg loss, 0.98 avg gain/loss, -0.52 EV
All, Buy and Hold 30 bars:
 22848 trades, 45.54% win pct, 14.08% avg gain, -14.67% avg loss, 0.96 avg gain/loss, -1.58 EV
All, Buy and Hold 60 bars:
 22848 trades, 44.82% win pct, 18.24% avg gain, -18.96% avg loss, 0.96 avg gain/loss, -2.29 EV
All, Buy and Hold:
 22848 trades, 43.08% win pct, 29.77% avg gain, -30.46% avg loss, 0.98 avg gain/loss, -4.52 EV
All, Buy and Hold 5 bars with Stop Loss of 5.00%:
 22848 trades, 47.19% win pct, 7.44% avg gain, -5.75% avg loss, 1.29 avg gain/loss, 0.47 EV
All, Buy and Hold 15 bars with Stop Loss of 5.00%:
 22848 trades, 38.70% win pct, 10.97% avg gain, -7.01% avg loss, 1.57 avg gain/loss, -0.05 EV
All, Buy and Hold 30 bars with Stop Loss of 5.00%:
 22848 trades, 31.06% win pct, 14.69% avg gain, -7.40% avg loss,

Gap ups - top gainers, Buy and Hold with Stop Loss of 5.00%:
 4710 trades, 17.39% win pct, 32.96% avg gain, -7.49% avg loss, 4.40 avg gain/loss, -0.46 EV


In [11]:
tradeSummaries |> highlightStrategies 10

Top 10 by: EV
Gap ups - new highs, Buy and Hold: 1.822461023902737286555059309
Top Gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.8367691275013426500254850525
New Highs, Buy and Hold: 0.736784720611110804648903637
Gap ups - top gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.7342425861245473845493995603
Gap ups, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.6082295648889924875236600645
Top Gainers, Buy and Hold 5 bars: 0.5825484022408140426570786786
Gap ups - top gainers, Buy and Hold 5 bars: 0.5698105391586863628178409871
Gap ups - top gainers, Buy and Hold 15 bars with Stop Loss of 5.00%: 0.5252717112219428407362557718
Gap ups, Buy and Hold 5 bars: 0.4783539464164832646621860451
All, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.4739211513203939924854011225


Top 10 by: Win %
Top Gainers, Buy and Hold 5 bars: 0.5171658660638598474145238768
Gap ups - top gainers, Buy and Hold 5 bars: 0.5154989384288747346072186837
Gap ups, Buy and Hold 5 bars: 0.50705754614549402823

In [17]:
Plotly.NET.Defaults.DefaultWidth <- 900

let metrics = [
    ("Win %", fun (s:studies.Types.TradeSummary) -> s.WinPct)
    ("Avg Win", fun s -> s.AvgWin)
    ("Avg Loss", fun s -> s.AvgLoss)
    ("Win/Loss", fun s -> s.AvgGainLoss)
    ("EV", fun s -> s.EV)
]

metrics
|> List.iter( fun (label,metricFunc) ->

    printfn $"sorting for {label}..."
    
    let sorted = tradeSummaries |> Seq.sortByDescending metricFunc |> Seq.toList
    
    let values = sorted |> List.map metricFunc
    let keys = sorted |> List.map (fun summary -> summary.StrategyName)

    printfn $"mapped keys and values for {label}, charting..."
    
    values |> List.zip keys |> List.iter (fun (l,v) -> printfn $"{l}: {v}")
    
    //let column = 
    //    Chart.Column(values = values, Keys = keys)
    //    |> Chart.withYAxisStyle (TitleText = label)
    
    // column.Display() |> ignore
    
    printfn ""
)

sorting for Win %...
mapped keys and values for Win %, charting...
Top Gainers, Buy and Hold 5 bars: 0.5171658660638598474145238768
Gap ups - top gainers, Buy and Hold 5 bars: 0.5154989384288747346072186837
Gap ups, Buy and Hold 5 bars: 0.507057546145494028230184582
All, Buy and Hold 5 bars: 0.5028448879551820728291316527
Gap ups - top gainers, Buy and Hold 15 bars: 0.5016985138004246284501061571
Gap ups, Buy and Hold 15 bars: 0.497440670079106561191251745
Gap ups - top gainers, Buy and Hold 60 bars: 0.4912951167728237791932059448
New Highs, Buy and Hold: 0.4899907961343764381040036815
Gap ups - new highs, Buy and Hold 15 bars: 0.4858952216465169833045480714
Gap ups - top gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.4855626326963906581740976645
Gap ups - new highs, Buy and Hold 5 bars: 0.4841681059297639608520437536
Gap ups - new highs, Buy and Hold: 0.4841681059297639608520437536
Gap ups, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.4834806886924150767798976268
Gap ups, B

Top Gainers, Buy and Hold 5 bars: 9.012577175175520860604430209
Gap ups - top gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 7.6054216904664421835744104171
Gap ups - top gainers, Buy and Hold 5 bars: 7.5353517645271833467407050589
All, Buy and Hold 5 bars with Stop Loss of 5.00%: 7.4369635815836703741350333123
All, Buy and Hold 5 bars: 7.4024309746557244026484507834
New Highs, Buy and Hold 15 bars with Stop Loss of 5.00%: 6.9733284711871417438373737074
Gap ups - new highs, Buy and Hold 15 bars with Stop Loss of 5.00%: 6.8880689956415042274259047115
Gap ups, Buy and Hold 5 bars with Stop Loss of 5.00%: 6.8861470474956621747254663856
Gap ups, Buy and Hold 5 bars: 6.8593339976116959844179781349
New Highs, Buy and Hold 15 bars: 6.8411780834560730644589447444
Gap ups - new highs, Buy and Hold 15 bars: 6.7411201894539561550342623892
Gap ups - new highs, Buy and Hold 5 bars: 4.9076441782647241464636606795
Gap ups - new highs, Buy and Hold 5 bars with Stop Loss of 5.00%: 4.9042420975267

All, Buy and Hold 5 bars with Stop Loss of 5.00%: 1.2940469790048549201866616171
Gap ups - new highs, Buy and Hold: 1.2635946275090736548676078349
Gap ups - new highs, Buy and Hold 15 bars with Stop Loss of 5.00%: 1.2557036533958876738240086743
Gap ups - new highs, Buy and Hold 5 bars with Stop Loss of 5.00%: 1.2335111137090521289907205354
New Highs, Buy and Hold 15 bars with Stop Loss of 5.00%: 1.2039214989459206793697561365
Gap ups - new highs, Buy and Hold 5 bars: 1.1807839066612831050592861028
Gap ups, Buy and Hold 5 bars: 1.1271896768135483314382913946
New Highs, Buy and Hold: 1.1132156498600392497630170687
Gap ups, Buy and Hold: 1.1052089662949335264823380469
Gap ups - top gainers, Buy and Hold 5 bars: 1.1014375864281482104562216786
Gap ups - top gainers, Buy and Hold: 1.08189696591307322223804643
New Highs, Buy and Hold 5 bars with Stop Loss of 5.00%: 1.077806225488367303245382497
All, Buy and Hold 5 bars: 1.0769226257176735918426548519
Top Gainers, Buy and Hold 5 bars: 1.066969